In [40]:
import sys
import os
lib = r'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\utilities\droplet_dataset'
lib2 = r'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\utilities'
lib3 = r'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\data_analysis'
lib4 = r'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy'
lib5 = r'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\scripts'
sys.path.append(lib)
sys.path.append(lib2)
sys.path.append(lib3)
sys.path.append(lib4)
sys.path.append(lib5)

In [41]:
import sklearn
from droplet_dataset import *
from utilities import *
from matplotlib import pyplot
import pandas as pd
import os.path as path
from classifying_cell_types import *

PROJECT_PATH = r'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy'

## Step 1: Extract sample from PC 

In [42]:
# Extracts one of the samples from PC
sample_id = 'M123'
sample_path = fr'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\Data\rna_seq200k\all_samples\{sample_id}\RNA_sample.pkl'
rna_sample = extract_droplet_data_from_pickle(sample_path)
rna_sample.counts = rna_sample.counts.T # TODO: fix that in a way that the RNA sample would be in the right shape in advance (cells are rows, genes are cols).
print(f'count shape {rna_sample.counts.shape}')
print(f'number of cells {rna_sample.number_of_cells}')
print(f'number of genes {rna_sample.number_of_genes}')

count shape (6743, 23673)
number of cells 6743
number of genes 23673


# Step 2: Builds positive/negative cell type marker table.


In [43]:
# Extract ImmuneCellsMarkersUpdated Excel file from PC and load it into DataFrame.
xls = pd.ExcelFile(MARKERS_PATH)
positive_markers_df = pd.read_excel(xls, 'and_or')
negative_markers_df = pd.read_excel(xls, 'none')
positive_markers_table = builds_cell_type_markers_table(positive_markers_df)
negative_markers_table = builds_cell_type_markers_table(negative_markers_df)

# * After this step negative/positive_markers_table object should map between each cell-index in extracted-sample's count-table and cell-type before elimination by negative markers
# * Next step is to eliminate cell-types by 'None' tab in Excel file and then analays results.

# Step 3: maps cell-type markers to sample's count-table cells.

In [44]:
positive_cell_types_mapping_table = maps_positive_cell_types_to_count_table(positive_markers_table, rna_sample)
negative_cell_types_mapping_table = maps_negative_cell_types_to_count_table(negative_markers_table, rna_sample)

T cells
['CD3E']
(311,)
CD4 helper T cells
['CD3E', 'CD4']
(86,)
CD8 Cytotoxic T cells
['CD3E', 'CD8A;CD8B']
(28,)
(29,)
Regulatory T cells
['CD3E', 'FOXP3', 'CTLA4;IL2RA']
(18,)
(12,)
Regulatory CD4 T cells
['CD4', 'CD3E', 'FOXP3', 'CTLA4;IL2RA']
(7,)
(2,)
Regulatory CD8 T cells
['CD3E', 'FOXP3', 'CTLA4;IL2RA', 'CD8A;CD8B']
(0,)
(1,)
(0,)
(0,)
Regulatory CD4_CD8 T cells
['CD3E', 'FOXP3', 'CTLA4;IL2RA', 'CD4', 'CD8A;CD8B']
(0,)
(0,)
(0,)
(0,)
NKT cells
['CD3E', 'NCR1;NCAM1']
(1,)
(2,)
NK cells
['FCGR3A', 'NCR1;NCAM1']
(0,)
(1,)
B cells
['CD19;MS4A1']
(61,)
(28,)
Activated T cells
['CD3E', 'CD2', 'CD28', 'IL2RA;CD69;ICOS;TNFRSF4;TNFRSF9;CD27', 'IL2;TNF;IFNG', 'CD8A;CD8B;CD4']
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(1,)
(0,)
(1,)
(0,)
(1,)
(0,)
(1,)
(0,)
(1,)
(0,)
(0,)
(1,)
(1,)
(2,)
(2,)
(1,)
(2,)
(0,)
(1,)
(0,)
(1,)
(2,)
(2,)
Senescence T cells
['CD3E', 'KLRG1', 'B3GAT1',

# Step 4: Cross-checks determinations between neg-markers and pos-markers and finds conflicts.

In [45]:
cross_check_mapping(rna_sample, positive_cell_types_mapping_table, negative_cell_types_mapping_table)

Number of cell with overlap between pos and neg markers: 181
number of classified cells: 611
number of cells (in sample): 6743
portion of classified cells in sample: 0.09061248702358


# Debug zone

In [88]:
# immune_genes = ['PTPRC', 'CD3E', 'CD8A', 'CD8B', 'CD4', 'MS4A1', 'CD163', 'CD14', 'CD19', 'CD2', 'NCR1', 'CD1C'] # immune general
# macrophage_genes = ['CD14', 'CD163'] # macrophages
# DC_genes = ['LILRA4', 'CD1C', 'CLE4C'] # dendritic cells
# lym_genes = ['CD3E','CD8A', 'CD8B', 'MS4A1', 'CD19','CD2', 'NCR1'] # lympocytes
# neutrophil_genes = ['FCGR3B', 'CXCR1','CXCR2','G0S2','CSF3R'] # neutrophils
# tumor_genes = ['MLANA', 'PMEL','TYR','MITF','AXL'] # tumor genes

# for gene in immune_genes:
#     if not gene in genes:
#         print(gene)
# immune_gene_indexes = [idx1 for idx1, g1 in enumerate(sample_genes) for idx2, g2 in enumerate(immune_genes) if g1==g2]
# sample_genes[immune_gene_indexes[0]]
# len(immune_gene_indexes)
# # counts.shape[:, immune_gene_indexes]
# interest_region = np.take(counts, immune_gene_indexes, axis=1)
# sum(np.sum(interest_region == 0, axis = 1)==0)
t_m = {'tumor':  ['MLANA', 'PMEL','TYR','MITF','AXL']}
# tumor_mapping = maps_negative_cell_types_to_count_table(t_m, rna_sample)
tumor_mapping

def finds_pos_neg_conflicts(positive_cell_types_mapping_table, negative_cell_types_mapping_table):
    conflict_indicators = np.zeros((len(positive_cell_types_mapping_table)), dtype=bool) # True if there is a conflict in place i, False otherwise
    for idx, pair in enumerate(zip(positive_cell_types_mapping_table.values(), negative_cell_types_mapping_table.values())):
        pos_classification = pair[0]
        neg_classification = pair[1]
        if len([cls for cls in pos_classification if cls in neg_classification])>0:
            conflict_indicators[idx] = True

            
def finds_cancer_conflicts(positive_cell_types_mapping_table, cancer_mapping_table):
    """
    Should be run after cleaning negative markers classification.
    """
    conflict_indicators = np.zeros((len(positive_cell_types_mapping_table)), dtype=bool) # True if there is a conflict in place i, False otherwise
    for idx, pair in enumerate(zip(positive_cell_types_mapping_table.values(), cancer_mapping_table.values())):
        pos_classification = pair[0]
        cancer_classification = pair[1]
        if len(pos_classification)>0 and len(cancer_classification)>0:
            conflict_indicators[idx] = True   
    return conflict_indicators


def cleans_by_indicators(positive_cell_types_mapping_table, conflict_indicators):
    conflict_indexes = np.where(conflict_indicators)
    for idx in conflict_indexes[0]:
        positive_cell_types_mapping_table[idx] = []
#     return positive_cell_types_mapping_table
            


summary_df = pd.DataFrame(columns=['sample name',
                                       'number of cells classified immune',
                                       'number of cells classified cancer',
                                       'number of pos-neg markers conflicts',
                                       'number of cancer-immune conflicts'])
summary_df.append(pd.DataFrame([[1,2,3,4, 6]], columns=summary_df.columns))

,sample name,number of cells classified immune,number of cells classified cancer,number of pos-neg markers conflicts,number of cancer-immune conflicts
0,1,2,3,4,6


In [96]:
# Debug zone!!!

# Extracts one of the samples from PC
sample_id = 'M100'
sample_path = fr'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\Data\rna_seq200k\all_samples\{sample_id}\RNA_sample.pkl'
rna_sample = extract_droplet_data_from_pickle(sample_path)
rna_sample.counts = rna_sample.counts.T # TODO: fix that in a way that the RNA sample would be in the right shape in advance (cells are rows, genes are cols).
print(sample_id)
print(f'count shape {rna_sample.counts.shape}')
print(f'number of cells {rna_sample.number_of_cells}')
print(f'number of genes {rna_sample.number_of_genes}')





tumor_genes = ['MLANA', 'PMEL','TYR','MITF','AXL'] # tumor genes
gran = ['CEACAM1', 'CEACAM1', 'CEACAM6', 'CEACAM3', 'ENPP3']
gran = ['CEACAM6']
curr_genes_indexes = find_indexes_of_markers_in_sample(rna_sample.gene_names, gran)

# curr_genes_indexes = find_indexes_of_markers_in_sample(rna_sample.gene_names, curr_markers)
counts_only_interesting_genes = np.take(rna_sample.counts, curr_genes_indexes, axis=1)
cells_satisfying_markers = np.all(counts_only_interesting_genes > 0, axis=1)
print(cells_satisfying_markers)
cells_satisfying_markers_indexs = np.where(cells_satisfying_markers)
n_cancer =cells_satisfying_markers_indexs[0].shape[0]
print(f'number of cancer cells: {n_cancer}')
print(f'percetage cancer cells: {n_cancer/rna_sample.number_of_cells}')
counts_only_interesting_genes

M100
count shape (4055, 20765)
number of cells 4055
number of genes 20765
[ True  True  True ...  True  True  True]
number of cancer cells: 4055
percetage cancer cells: 1.0


array([], shape=(4055, 0), dtype=uint16)

In [ ]:
# Debug zone!!!

# ROOT_PATH = r'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\Data\rna_seq200k\all_samples'
df_doc = pd.DataFrame(columns=['file', 'number of cells', 'number of classified cells', 'portion in percenatge'])
folders = [subfolder for subfolder in os.listdir(ROOT_PATH)]
folders
for sample_id in folders:
    print(sample_id)
    sample = fr'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\Data\rna_seq200k\all_samples\{sample_id}\RNA_sample.pkl'
    data = pickle.load(open(sample, 'rb'))
    sample_genes = data.gene_names
    counts = data.counts.T
    n_of_cells = counts.shape[0]
    n_of_genes = counts.shape[0]
#     print(f'count shape {counts.shape}')
    print(f'number of cells {n_of_cells}')
#     print(f'number of genes {n_of_genes}')
    len([v for v in positive_cell_types_mapping_table.values() if len(v)>0])

    curr_genes_indexes = find_indexes_of_markers_in_sample(sample_genes, ['CD3E', 'CD163', 'FUT4'])
    counts_only_interesting_genes = np.take(counts, curr_genes_indexes, axis=1)
    cells_satisfying_markers = np.any(counts_only_interesting_genes > 0, axis = 1)
    cells_satisfying_markers_indexs = np.where(cells_satisfying_markers)
    n_c_c = cells_satisfying_markers_indexs[0].shape[0]
    print(f'number of classified cells: {n_c_c}')
    df_doc = df_doc.append({'file':sample_id,  'number of cells':n_of_cells, 'number of classified cells':n_c_c, 'portion in percenatge':n_c_c/n_of_cells}, ignore_index=True)    # cells_satisfying_markers_indexs
    df_doc.to_csv(r'C:\Users\itay\Desktop\test\o.csv')